In [1]:
import pickle

import glob2
import datetime

import numpy as np
from utils.data_reading.sound_data.station import StationsCatalog

In [2]:
datasets_yaml = "/home/plerolland/Bureau/dataset.yaml"
year = 2012
stations_c = StationsCatalog(datasets_yaml).filter_out_undated().filter_out_unlocated()

# output
detections_file = f"../../data/detections/{year}/detections.npy"

In [3]:
to_read = glob2.glob(f"../../data/detections/{year}/*.p")

detections = {}
for file in to_read:
    info = file.split("_")[-1][:-2].split("-")
    name, year_station = "-".join(info[:-1]), int(info[-1])
    station = stations_c.by_starting_year(year_station).by_names(name)[0]
    detections[station] = []
    with open(file, "rb") as f:
        while True:
            try:
                detections[station].append(pickle.load(f))
            except EOFError:
                break
                
    detections[station] = np.array(detections[station])
    detections[station] = detections[station][np.argsort(detections[station][:,0])]

    # merge close detections
    to_del = set()
    i, j = 0, 0
    while j<len(detections[station]):
        delta = (np.abs(detections[station][i,0] - detections[station][j,0]))
        if delta.total_seconds() < 5:
            detections[station][i,0] += delta / 2
            to_del.add(j)
        else:
            i = j
        j+=1
    detections[station] = np.delete(detections[station], list(to_del), axis=0)
    
    print(f"{len(list(to_del))} lines deleted ; {len(detections[station])} left for station {station}")
np.save(detections_file, detections)

7 lines deleted ; 192469 left for station station_MAD_2012
42 lines deleted ; 401820 left for station station_NCRO3_2012
15 lines deleted ; 365641 left for station station_NEAMS_2012
1 lines deleted ; 28280 left for station station_RAMA_2012
6 lines deleted ; 165665 left for station station_SWAMS_2012
6 lines deleted ; 359998 left for station station_WKER1_2012
37 lines deleted ; 335628 left for station station_WKER2_2012
17 lines deleted ; 513066 left for station station_WKER3_2012
3 lines deleted ; 58331 left for station station_MAD_2011
7 lines deleted ; 42711 left for station station_NCRO2_2011
1 lines deleted ; 6951 left for station station_NCRO3_2011
4 lines deleted ; 56984 left for station station_WKER1_2011
18 lines deleted ; 398280 left for station station_H01W1_2012
4 lines deleted ; 141976 left for station station_H08N1_2012
7 lines deleted ; 202738 left for station station_H08S1_2012
